- 데이터셋의 형식: csv(값을 쉼표로 구분), tsv(값을 탭으로 구분), xlsx(액셀 통합 문서) 등
  - 일부 데이터 형식은 데이터를 표 형식으로 저장하지 않음 -> 키-값 형식으로 관련된 데이터 컬렉션을 중첩하여 저장
  - 판다스는 키-값 데이터 <-> 테이블 형식의 데이터 로 저장하는 기능을 제공

# JSON 파일 읽고 쓰기
- JavaScript Object Notation: 텍스트 데이터를 저장하고 전송하기 위한 키-값 저장 형식
- JSON은 JavaScript에서 구문적으로 영감을 받았지만, **언어 독립적**: 대부부느이 언어는 JSON을 생성하고 파싱 가능
- JSON 응답: 키-값 쌍으로 구성, 키가 값의 고유 식별자 역할, 콜론 기호로 연결, 키는 문자열이어야함
- JSON은 웹사이트 서버와 같은 최신 API에 널리 사용되는 응답 형식
- 린터(linter): 키-값 쌍을 별도의 줄에 배치하여 JSON 응답을 가독성 있는 형식으로 나타내는 소프트웨어 프로그램
```
{
    "name": "Harry Potter",
    "age": 17,
    "wizard": true,
    "friends": ["Ron Weasley", "Hermione Granger"],
    "address": {
        "street": "4 Privet Drive",
        "town": "Little Whinging"
    }
}
```
## JSON 파일을 DataFrame으로 불러오기

In [1]:
#read_json 메서드
import pandas as pd
nobel = pd.read_json("nobel.json")
nobel.head()

,prizes
0,"{'year': '2019', 'category': 'chemistry', 'lau..."
1,"{'year': '2019', 'category': 'economics', 'lau..."
2,"{'year': '2019', 'category': 'literature', 'la..."
3,"{'year': '2019', 'category': 'peace', 'laureat..."
4,"{'year': '2019', 'category': 'physics', 'overa..."


In [2]:
#판다스는 JSON을 가져올때 최상위 prize 키를 열의 이름으로 설정 후
#JSON에서 파싱한 각 키-값 쌍에 대한 파이썬 딕셔너리를 생성
nobel.loc[2, "prizes"]

{'year': '2019',
 'category': 'literature',
 'laureates': [{'id': '980',
   'firstname': 'Peter',
   'surname': 'Handke',
   'motivation': '"for an influential work that with linguistic ingenuity has explored the periphery and the specificity of human experience"',
   'share': '1'}]}

In [3]:
type(nobel.loc[2, "prizes"])

dict

- JSON의 최상위 키-값 쌍을 추출하여 DataFrame 열로 분리,
- 값이 리스트인 경우, 각 딕셔너리를 순회하며 중첩 정보를 추출
- **평탄화(flattering)/정규화(normalizing)**: 중첩된 데이터 레코드를 단일 1차원 리스트로 변형하는 과정
  - 판다스에 있는 json_normalize 함수는 평탄화(정규화) 작업을 처리

In [4]:
chemistry_2019 = nobel.loc[0, "prizes"]
chemistry_2019

{'year': '2019',
 'category': 'chemistry',
 'laureates': [{'id': '976',
   'firstname': 'John',
   'surname': 'Goodenough',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '977',
   'firstname': 'M. Stanley',
   'surname': 'Whittingham',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '978',
   'firstname': 'Akira',
   'surname': 'Yoshino',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'}]}

In [5]:
#json_normalize 함수의 data 매개변수에 딕셔너리를 전달
pd.json_normalize(data = chemistry_2019)

,year,category,laureates
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':..."


In [6]:
#json_normalize 함수의 record_path 매개변수로 중첩된 레코드를 정규화 가능
pd.json_normalize(data = chemistry_2019, record_path = "laureates")

,id,firstname,surname,motivation,share
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3


In [7]:
#json_normalize 함수의 meta 매개변수로 최상위 키-값 쌍을 유지
pd.json_normalize(
    data = chemistry_2019,
    record_path = "laureates",
    meta = ["year", "category"]
)

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


In [8]:
#json_normalize 함수는 data 매개변수에 딕셔너리 Series를 받아서 같은 일을 처리 가능
pd.json_normalize(
    data = nobel["prizes"],
    record_path = "laureates",
    meta = ["year", "category"]
)

KeyError: "Key 'laureates' not found. If specifying a record_path, all elements of data should have the path."

In [9]:
#prizes Series에 있는 딕셔너리 중 일부는 "laureates"라는 키가 없어서 KeyError 발생
#따라서 "laureates" 키가 없는 딕셔너리를 식별하고 수동으로 키-값(빈 리스트) 할당
cheese_consumption = {
    "France": 57.9,
    "Germany": 53.2,
    "Luxembourg": 53.2
}

In [10]:
#setdefault 메서드: 딕셔너리에 키가 없는 경우 키-값 쌍을 할당
#키가 있는 경우 기존 값을 반환
#첫번째 인수는 키, 두번째 인수는 값
cheese_consumption.setdefault("France", 100)

57.9

In [11]:
cheese_consumption["France"]

57.9

In [12]:
#키가 없는 경우, 키를 추가하는 예제
cheese_consumption.setdefault("Italy", 48)

48

In [13]:
cheese_consumption

{'France': 57.9, 'Germany': 53.2, 'Luxembourg': 53.2, 'Italy': 48}

In [14]:
#prizes에 있는 중첩 딕셔너리에 적용 
#-> apply 메서드를 활용하여 Series의 요소를 개별적으로 순회
#setdefault 메서드는 딕셔너리 자체를 변경하므로 덮어쓸 필요 없음
def add_laureates_key(entry):
    entry.setdefault("laureates", [])

nobel["prizes"].apply(add_laureates_key)

0      None
1      None
2      None
3      None
4      None
       ... 
641    None
642    None
643    None
644    None
645    None
Name: prizes, Length: 646, dtype: object

In [15]:
winners = pd.json_normalize(
    data = nobel["prizes"],
    record_path = "laureates",
    meta = ["year", "category"]
)
winners

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry
3,982,Abhijit,Banerjee,"""for their experimental approach to alleviatin...",3,2019,economics
4,983,Esther,Duflo,"""for their experimental approach to alleviatin...",3,2019,economics
...,...,...,...,...,...,...,...
945,569,Sully,Prudhomme,"""in special recognition of his poetic composit...",1,1901,literature
946,462,Henry,Dunant,"""for his humanitarian efforts to help wounded ...",2,1901,peace
947,463,Frédéric,Passy,"""for his lifelong work for international peace...",2,1901,peace
948,1,Wilhelm Conrad,Röntgen,"""in recognition of the extraordinary services ...",1,1901,physics


## DataFrame을 JSON 파일로 내보내기
- to_json 메서드: 판다스 자료구조에서 JSON 문자열을 생성 (**덮어씀**)
  - orient 매개변수: 판다스가 데이터를 변환하는 형식을 사용자 정의
  

In [16]:
winners.head(2)

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry


In [18]:
#orient 매개변수에 인수로 records 전달: 키-값 객체로 구성된 JSON 배열을 반환
winners.head(2).to_json(orient = "records")

'[{"id":"976","firstname":"John","surname":"Goodenough","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"},{"id":"977","firstname":"M. Stanley","surname":"Whittingham","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"}]'

In [19]:
#orient 매개변수에 인수로 split을 전달: 
#인덱스와 데이터 키가 별도로 있는 딕셔너리를 반환(각 행의 항목에 대한 열 이름 중복 방지)
winners.head(2).to_json(orient = "split")

'{"columns":["id","firstname","surname","motivation","share","year","category"],"index":[0,1],"data":[["976","John","Goodenough","\\"for the development of lithium-ion batteries\\"","3","2019","chemistry"],["977","M. Stanley","Whittingham","\\"for the development of lithium-ion batteries\\"","3","2019","chemistry"]]}'

- 이외에 orient 매개변수에 인수로 'index', 'columns', 'values', 'table'을 전달 가능

In [20]:
winners.to_json("winners.json", orient = "records")

# CSV 파일 읽고 쓰기
- read_csv 파일에 url을 첫번째 인수로 전달->판다스가 자동으로 데이터셋을 받아서 DataFrmae으로 가져옴
  - 하드코딩된 URL은 자주 변경되는 실시간 데이터를 분석해야 하는 경우 유용

In [23]:
url = "https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv"
baby_names = pd.read_csv(url)
baby_names.head()

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [25]:
#to_csv 메서드: 일반 CSV파일에 DataFrame을 작성(write)
#인수 없이 호출 시 CSV문자열을 주피터 노트북에 직접 출력
#판다스는 CSV문자열에 DataFrame 인덱스를 포함
baby_names.head(10).to_csv()

",Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank\r\n0,2011,FEMALE,HISPANIC,GERALDINE,13,75\r\n1,2011,FEMALE,HISPANIC,GIA,21,67\r\n2,2011,FEMALE,HISPANIC,GIANNA,49,42\r\n3,2011,FEMALE,HISPANIC,GISELLE,38,51\r\n4,2011,FEMALE,HISPANIC,GRACE,36,53\r\n5,2011,FEMALE,HISPANIC,GUADALUPE,26,62\r\n6,2011,FEMALE,HISPANIC,HAILEY,126,8\r\n7,2011,FEMALE,HISPANIC,HALEY,14,74\r\n8,2011,FEMALE,HISPANIC,HANNAH,17,71\r\n9,2011,FEMALE,HISPANIC,HAYLEE,17,71\r\n"

In [26]:
#to_csv 메서드의 index 매개변수에 인수로 False 전달: 인덱스 제외
baby_names.head(10).to_csv(index = False)

"Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank\r\n2011,FEMALE,HISPANIC,GERALDINE,13,75\r\n2011,FEMALE,HISPANIC,GIA,21,67\r\n2011,FEMALE,HISPANIC,GIANNA,49,42\r\n2011,FEMALE,HISPANIC,GISELLE,38,51\r\n2011,FEMALE,HISPANIC,GRACE,36,53\r\n2011,FEMALE,HISPANIC,GUADALUPE,26,62\r\n2011,FEMALE,HISPANIC,HAILEY,126,8\r\n2011,FEMALE,HISPANIC,HALEY,14,74\r\n2011,FEMALE,HISPANIC,HANNAH,17,71\r\n2011,FEMALE,HISPANIC,HAYLEE,17,71\r\n"

In [27]:
#to_csv 메서드의 첫번째 인수로 문자열을 전달해 파일 이름을 지정 가능
baby_names.to_csv("NYC_Baby_Names.csv", index = False)

In [28]:
#to_csv 메서드의 columns 매개변수에 이름을 담은 리스트를 전달하여 내보낼 열 지정 가능
baby_names.to_csv(
    "NYC_Baby_Names.csv",
    index = False,
    columns = ["Gender", "Child's First Name", "Count"]
)

# 엑셀 통합문서에 읽고 쓰기
## 아나콘다 환경에 xlrd와 openpyxl 라이브러리 설치
```
conda info --envs
conda activate base
conda install xlrd openpyxl
```
## 엑셀 통합문서 가져오기
- read_excel 함수
  - 판다스의 최상위 수준에 있음
  - 엑셀 통합문서를 DataFrame으로 가져옴
  - 첫번째 매개변수 io: 통합문서의 경로가 포함된 문자열(.xlsx 확장자 포함)

In [29]:
pd.read_excel("Single Worksheet.xlsx")

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [31]:
#read_excel 함수의 매개변수들 ~= read_csv의 매개변수
#index_col(인덱스 열 설정), cols(일반 열 설정), squeeze(1열 DataFrame을 Series 객체로 강제 변환)
pd.read_excel(
    io = "Single Worksheet.xlsx",
    usecols = ["City", "First Name", "Last Name"], 
    index_col = "City"#index_col을 사용하는 경우 usecols 리스트에도 해당 열 포함
)

,First Name,Last Name
City,,
Miami,Brandon,James
Denver,Sean,Hawkins
Los Angeles,Judy,Day
San Francisco,Ashley,Ruiz
Portland,Stephanie,Gomez


In [32]:
#엑셀 통합문서에 여러 개의 워크시트가 있는 경우 각 워크시트에 0부터 인덱스 위치를 할당
#기본값=0
pd.read_excel("Multiple Worksheets.xlsx")

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [33]:
#sheet_name 매개변수: 특정 워크시트를 가져올 수 있음
pd.read_excel("Multiple Worksheets.xlsx", sheet_name = 0)
#pd.read_excel("Multiple Worksheets.xlsx", sheet_name = "Data 1")

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [35]:
#모든 워크시트를 가져오려면 sheet_name에 None을 전달
#워크시트의 이름을 키로 하는 딕셔너리를 반환
workbook = pd.read_excel(
    "Multiple Worksheets.xlsx", sheet_name = None
)
workbook

{'Data 1':   First Name Last Name           City Gender
 0    Brandon     James          Miami      M
 1       Sean   Hawkins         Denver      M
 2       Judy       Day    Los Angeles      F
 3     Ashley      Ruiz  San Francisco      F
 4  Stephanie     Gomez       Portland      F,
 'Data 2':   First Name Last Name           City Gender
 0     Parker     Power        Raleigh      F
 1    Preston  Prescott   Philadelphia      F
 2    Ronaldo   Donaldo         Bangor      M
 3      Megan   Stiller  San Francisco      M
 4     Bustin    Jieber         Austin      F,
 'Data 3':   First Name  Last Name     City Gender
 0     Robert     Miller  Seattle      M
 1       Tara     Garcia  Phoenix      F
 2    Raphael  Rodriguez  Orlando      M}

In [36]:
type(workbook)

dict

In [38]:
workbook["Data 2"]

,First Name,Last Name,City,Gender
0,Parker,Power,Raleigh,F
1,Preston,Prescott,Philadelphia,F
2,Ronaldo,Donaldo,Bangor,M
3,Megan,Stiller,San Francisco,M
4,Bustin,Jieber,Austin,F


In [40]:
#가져올 워크시트의 하위 집합을 지정 -> 리스트를 sheet_name 매개변수에 전달
pd.read_excel(
    "Multiple Worksheets.xlsx",
    sheet_name = ["Data 1", "Data 3"]
)

{'Data 1':   First Name Last Name           City Gender
 0    Brandon     James          Miami      M
 1       Sean   Hawkins         Denver      M
 2       Judy       Day    Los Angeles      F
 3     Ashley      Ruiz  San Francisco      F
 4  Stephanie     Gomez       Portland      F,
 'Data 3':   First Name  Last Name     City Gender
 0     Robert     Miller  Seattle      M
 1       Tara     Garcia  Phoenix      F
 2    Raphael  Rodriguez  Orlando      M}

## 엑셀 통합문서 내보내기

In [41]:
baby_names.head()

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [42]:
girls = baby_names[baby_names["Gender"] == "FEMALE"]
boys = baby_names[baby_names["Gender"] == "MALE"]

In [43]:
#ExcelWriter 객체 생성
#ExcelWriter 생성자는 판다스 라이브러리 최상위 속성으로 접근 가능
#첫번째 매개변수 path: 새 통합문서의 파일 이름을 문자열로 전달 받음
excel_file = pd.ExcelWriter("Baby_Names.xlsx")
excel_file

In [44]:
#to_excel 메서드: 엑셀 통합문서에 쓰는 DataFrame의 메서드
#첫번째 매개변수 excel_writer: ExcelWriter 객체를 전달받음
#sheet_name 매개변수: 워크시트의 이름을 문자열로 받음
#index 매개변수: DataFrame의 인덱스를 포함할지 여부
girls.to_excel(
    excel_writer = excel_file, sheet_name = "Girls", index = False
)

In [45]:
#column 매개변수에 사용자 정의 리스트를 전달하여 하위 집합만 내보낼 수 있음
boys.to_excel(
    excel_file,
    sheet_name = "Boys",
    index = False,
    columns = ["Child's First Name", "Count", "Rank"]
)

In [47]:
#ExcelWriter 객체에서 save 메서드를 호출하여 최종 결과 도출
excel_file.close()

# 코딩 챌린지

In [48]:
tv_shows_json = pd.read_json("tv_shows.json")
tv_shows_json

,shows
0,"{'show': 'The X-Files', 'runtime': 60, 'networ..."
1,"{'show': 'Lost', 'runtime': 60, 'network': 'AB..."
2,"{'show': 'Buffy the Vampire Slayer', 'runtime'..."


In [49]:
tv_shows_json.loc[0, "shows"]

{'show': 'The X-Files',
 'runtime': 60,
 'network': 'FOX',
 'episodes': [{'season': 1,
   'episode': 1,
   'name': 'Pilot',
   'air_date': '1993-09-11 01:00:00'},
  {'season': 1,
   'episode': 2,
   'name': 'Deep Throat',
   'air_date': '1993-09-18 01:00:00'},
  {'season': 1,
   'episode': 3,
   'name': 'Squeeze',
   'air_date': '1993-09-25 01:00:00'},
  {'season': 1,
   'episode': 4,
   'name': 'Conduit',
   'air_date': '1993-10-02 01:00:00'},
  {'season': 1,
   'episode': 5,
   'name': 'The Jersey Devil',
   'air_date': '1993-10-09 01:00:00'},
  {'season': 1,
   'episode': 6,
   'name': 'Shadows',
   'air_date': '1993-10-23 01:00:00'},
  {'season': 1,
   'episode': 7,
   'name': 'Ghost in the Machine',
   'air_date': '1993-10-30 01:00:00'},
  {'season': 1,
   'episode': 8,
   'name': 'Ice',
   'air_date': '1993-11-06 02:00:00'},
  {'season': 1,
   'episode': 9,
   'name': 'Space',
   'air_date': '1993-11-13 02:00:00'},
  {'season': 1,
   'episode': 10,
   'name': 'Fallen Angel',
   '

In [52]:
#1. shows열의 각 딕셔너리에 중첩된 에피소드 데이터를 정규화, 
#각 에피소드를 별도의 행으로 나타내는 DataFrame을 생성
tv_shows = pd.json_normalize(
    data = tv_shows_json["shows"],
    record_path = "episodes",
    meta = ["show", "runtime", "network"]
)
tv_shows

,season,episode,name,air_date,show,runtime,network
0,1,1,Pilot,1993-09-11 01:00:00,The X-Files,60,FOX
1,1,2,Deep Throat,1993-09-18 01:00:00,The X-Files,60,FOX
2,1,3,Squeeze,1993-09-25 01:00:00,The X-Files,60,FOX
3,1,4,Conduit,1993-10-02 01:00:00,The X-Files,60,FOX
4,1,5,The Jersey Devil,1993-10-09 01:00:00,The X-Files,60,FOX
...,...,...,...,...,...,...,...
477,7,18,Dirty Girls,2003-04-16 00:00:00,Buffy the Vampire Slayer,60,UPN
478,7,19,Empty Places,2003-04-30 00:00:00,Buffy the Vampire Slayer,60,UPN
479,7,20,Touched,2003-05-07 00:00:00,Buffy the Vampire Slayer,60,UPN
480,7,21,End of Days,2003-05-14 00:00:00,Buffy the Vampire Slayer,60,UPN


In [54]:
#2. 정규화된 데이터셋을 각 프로그램별로 하나씩 총 3개의 개별 DataFrame으로 필터링
xfiles = tv_shows[tv_shows["show"] == "The X-Files"]
lost = tv_shows[tv_shows["show"] == "Lost"]
buffy = tv_shows[tv_shows["show"] == "Buffy the Vampire Slayer"]

In [55]:
#3. 3개의 DataFrame을 episodes.xlsx 엑셀 통합문서로 작성, 에피소드별로 별도의 워크시트에 저장
episodes = pd.ExcelWriter("episodes.xlsx")
xfiles.to_excel(
    excel_writer = episodes, sheet_name = "X_Files", index = False
)
lost.to_excel(
    excel_writer = episodes, sheet_name = "Lost", index = False
)
buffy.to_excel(
    excel_writer = episodes, sheet_name = "Buffy the Vampire Slayer", index = False
)
episodes.close()